In [1]:
import tensorflow as tf
import numpy as np
import random
import gym
import os

In [2]:
def policy_gradient():
    with tf.variable_scope("policy"):
        params = tf.get_variable("policy_parameters", [4,2])
        state = tf.placeholder("float", [None, 4], name="state")
        # NOTE: have to specify shape of actions so we can call
        # get_shape when calculating g_log_prob below
        actions = tf.placeholder("float", [200, 2], name="actions")
        advantages = tf.placeholder("float", [None,], name="advantages")
        linear = tf.matmul(state, params)
        probabilities = tf.nn.softmax(linear)
        my_variables = tf.trainable_variables()

        # calculate the probability of the chosen action given the state
        action_log_prob = tf.log(tf.reduce_sum(
            tf.multiply(probabilities, actions), reduction_indices=[1]))

        # calculate the gradient of the log probability at each point in time
        # NOTE: doing this because tf.gradients only returns a summed version
        action_log_prob_flat = tf.reshape(action_log_prob, (-1,))

        g_log_prob = tf.stack(
            [tf.gradients(action_log_prob_flat[i], my_variables)[0]
                for i in range(action_log_prob_flat.get_shape()[0])])
        g_log_prob = tf.reshape(g_log_prob, (200, 8, 1))

        # calculate the policy gradient by multiplying by the advantage function
        g = tf.multiply(g_log_prob, tf.reshape(advantages, (200, 1, 1)))
        # sum over time
        g = 1.00 / 200.00 * tf.reduce_sum(g, reduction_indices=[0])

        # calculate the Fischer information matrix and its inverse
        F2 = tf.map_fn(lambda x: tf.matmul(x, tf.transpose(x)), g_log_prob)
        F = 1.0 / 200.0 * tf.reduce_sum(F2, reduction_indices=[0])

        # calculate inverse of positive definite clipped F
        # NOTE: have noticed small eigenvalues (1e-10) that are negative,
        # using SVD to clip those out, assuming they're rounding errors
        S, U, V = tf.svd(F)
        atol = tf.reduce_max(S) * 1e-6
        S_inv = tf.divide(1.0, S)
        S_inv = tf.where(S < atol, tf.zeros_like(S), S_inv)
        S_inv = tf.diag(S_inv)
        F_inv = tf.matmul(S_inv, tf.transpose(U))
        F_inv = tf.matmul(V, F_inv)

        # calculate natural policy gradient ascent update
        F_inv_g = tf.matmul(F_inv, g)
        # calculate a learning rate normalized such that a constant change
        # in the output control policy is achieved each update, preventing
        # any parameter changes that hugely change the output
        learning_rate = tf.sqrt(
            tf.divide(0.001, tf.matmul(tf.transpose(g), F_inv_g)))

        update = tf.multiply(learning_rate, F_inv_g)
        update = tf.reshape(update, (4, 2))

        # update trainable parameters
        # NOTE: whenever my_variables is fetched they're also updated
        my_variables[0] = tf.assign_add(my_variables[0], update)

        return probabilities, state, actions, advantages, my_variables

In [3]:
def value_gradient():
    with tf.variable_scope("value"):
        state = tf.placeholder("float", [None, 4])
        newvals = tf.placeholder("float", [None, 1])
        w1 = tf.get_variable("w1", [4, 10])
        b1 = tf.get_variable("b1", [10])
        h1 = tf.nn.relu(tf.matmul(state,w1) + b1)
        w2 = tf.get_variable("w2", [10, 1])
        b2 = tf.get_variable("b2", [1])
        calculated = tf.matmul(h1, w2) + b2

        # minimize the difference between predicted and actual
        diffs = calculated - newvals
        loss = tf.nn.l2_loss(diffs)
        optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)

        return calculated, state, newvals, optimizer, loss


In [4]:
def run_episode(env, policy_grad, value_grad, sess):
    # unpack the policy network (generates control policy)
    (pl_calculated, pl_state, pl_actions,
        pl_advantages, pl_optimizer) = policy_grad
    # unpack the value network (estimates expected reward)
    (vl_calculated, vl_state, vl_newvals,
        vl_optimizer, vl_loss) = value_grad

    # set up the environment
    observation = env.reset()

    episode_reward = 0
    total_rewards = []
    states = []
    actions = []
    advantages = []
    transitions = []
    update_vals = []

    n_episodes = 0
    n_timesteps = 200
    for t in range(n_timesteps):
        # calculate policy
        obs_vector = np.expand_dims(observation, axis=0)
        probs = sess.run(
            pl_calculated,
            feed_dict={pl_state: obs_vector})

        # stochastically generate action using the policy output
        action = 0 if random.uniform(0,1) < probs[0][0] else 1
        # record the transition
        states.append(observation)
        actionblank = np.zeros(2)
        actionblank[action] = 1
        actions.append(actionblank)
        # take the action in the environment
        old_observation = observation
        observation, reward, done, info = env.step(action)
        transitions.append((old_observation, action, reward))
        episode_reward += reward

        # if the pole falls or time is up
        if done or t == n_timesteps - 1:
            for ii, trans in enumerate(transitions):
                obs, action, reward = trans

                # calculate discounted monte-carlo return
                future_reward = 0
                future_transitions = len(transitions) - ii
                decrease = 1
                for jj in range(future_transitions):
                    future_reward += transitions[jj + ii][2] * decrease
                    decrease = decrease * 0.97
                obs_vector = np.expand_dims(obs, axis=0)
                # compare the calculated expected reward to the average
                # expected reward, as estimated by the value network
                currentval = sess.run(
                    vl_calculated, feed_dict={vl_state: obs_vector})[0][0]

                # advantage: how much better was this action than normal
                advantages.append(future_reward - currentval)

                # update the value function towards new return
                update_vals.append(future_reward)

            n_episodes += 1
            # reset variables for next episode in batch
            total_rewards.append(episode_reward)
            episode_reward = 0.0
            transitions = []

            if done:
                # if the pole fell, reset environment
                observation = env.reset()
            else:
                # if out of time, close environment
                env.close()

    print('total_rewards: ', total_rewards)

    # update value function
    update_vals_vector = np.expand_dims(update_vals, axis=1)
    sess.run(vl_optimizer,
             feed_dict={vl_state: states,
                        vl_newvals: update_vals_vector})
    # update control policy
    sess.run(pl_optimizer,
             feed_dict={pl_state: states,
                        pl_advantages: advantages,
                        pl_actions: actions})

    return total_rewards, n_episodes


In [5]:
# generate the networks
policy_grad = policy_gradient()
value_grad = value_gradient()

# run the training from scratch 10 times, record results
for ii in range(10):

    env = gym.make('CartPole-v0')
    env = gym.wrappers.Monitor(
        env=env,
        directory='cartpole-hill/',
        force=True,
        video_callable=False)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    
    if not os.path.exists('natural'):
        os.mkdir('natural')
    if not os.path.exists(os.path.join('natural','third')):
        os.mkdir(os.path.join('natural','third'))

    summ_writer_3 = tf.summary.FileWriter(os.path.join('natural','third'), sess.graph)


    max_rewards = []
    total_episodes = []
    # each batch is 200 time steps worth of episodes
    n_training_batches = 300
    import time
    times = []
    for i in range(n_training_batches):
        start_time = time.time()
        if i % 100 == 0:
            print(i)
        reward, n_episodes = run_episode(env, policy_grad, value_grad, sess)
        max_rewards.append(np.max(reward))
        total_episodes.append(n_episodes)
        times.append(time.time() - start_time)
    print('average time: %.3f' % (np.sum(times) / n_training_batches))

    #np.savez_compressed('data/natural_policy_gradient_%i' % ii,
    #        max_rewards=max_rewards, total_episodes=total_episodes)

    sess.close()

0
total_rewards:  [67.0, 22.0, 31.0, 63.0, 17.0]
total_rewards:  [25.0, 59.0, 22.0, 34.0, 60.0]
total_rewards:  [86.0, 14.0, 62.0, 38.0]
total_rewards:  [43.0, 27.0, 52.0, 69.0, 9.0]
total_rewards:  [49.0, 55.0, 84.0, 12.0]
total_rewards:  [72.0, 20.0, 27.0, 70.0, 11.0]
total_rewards:  [57.0, 86.0, 57.0]
total_rewards:  [73.0, 50.0, 77.0]
total_rewards:  [81.0, 51.0, 68.0]
total_rewards:  [54.0, 97.0, 49.0]
total_rewards:  [70.0, 72.0, 58.0]
total_rewards:  [65.0, 71.0, 64.0]
total_rewards:  [80.0, 70.0, 50.0]
total_rewards:  [91.0, 61.0, 22.0, 26.0]
total_rewards:  [73.0, 69.0, 58.0]
total_rewards:  [135.0, 65.0]
total_rewards:  [75.0, 73.0, 52.0]
total_rewards:  [98.0, 72.0, 30.0]
total_rewards:  [80.0, 120.0]
total_rewards:  [75.0, 59.0, 66.0]
total_rewards:  [70.0, 90.0, 40.0]
total_rewards:  [90.0, 81.0, 29.0]
total_rewards:  [127.0, 73.0]
total_rewards:  [47.0, 66.0, 87.0]
total_rewards:  [110.0, 65.0, 25.0]
total_rewards:  [73.0, 58.0, 69.0]
total_rewards:  [96.0, 57.0, 47.0]
to

total_rewards:  [40.0, 44.0, 51.0, 65.0]
total_rewards:  [69.0, 68.0, 37.0, 26.0]
total_rewards:  [31.0, 96.0, 73.0]
total_rewards:  [51.0, 86.0, 63.0]
total_rewards:  [65.0, 28.0, 62.0, 45.0]
total_rewards:  [73.0, 71.0, 56.0]
total_rewards:  [60.0, 119.0, 21.0]
total_rewards:  [63.0, 70.0, 60.0, 7.0]
total_rewards:  [79.0, 121.0]
total_rewards:  [49.0, 65.0, 78.0, 8.0]
total_rewards:  [90.0, 83.0, 27.0]
total_rewards:  [57.0, 137.0, 6.0]
total_rewards:  [66.0, 79.0, 55.0]
total_rewards:  [78.0, 79.0, 43.0]
total_rewards:  [94.0, 106.0]
total_rewards:  [89.0, 111.0]
total_rewards:  [108.0, 92.0]
total_rewards:  [77.0, 123.0]
total_rewards:  [145.0, 55.0]
total_rewards:  [168.0, 32.0]
total_rewards:  [103.0, 89.0, 8.0]
total_rewards:  [87.0, 86.0, 27.0]
total_rewards:  [108.0, 86.0, 6.0]
total_rewards:  [118.0, 82.0]
total_rewards:  [181.0, 19.0]
total_rewards:  [177.0, 23.0]
total_rewards:  [104.0, 96.0]
total_rewards:  [101.0, 99.0]
total_rewards:  [148.0, 52.0]
total_rewards:  [161.

total_rewards:  [17.0, 60.0, 26.0, 27.0, 13.0, 23.0, 34.0]
total_rewards:  [26.0, 17.0, 13.0, 18.0, 115.0, 11.0]
total_rewards:  [55.0, 18.0, 17.0, 31.0, 64.0, 15.0]
total_rewards:  [29.0, 20.0, 95.0, 49.0, 7.0]
total_rewards:  [27.0, 16.0, 45.0, 102.0, 10.0]
total_rewards:  [41.0, 71.0, 15.0, 55.0, 18.0]
total_rewards:  [37.0, 67.0, 33.0, 42.0, 21.0]
total_rewards:  [84.0, 55.0, 61.0]
total_rewards:  [112.0, 43.0, 45.0]
total_rewards:  [63.0, 71.0, 66.0]
total_rewards:  [115.0, 34.0, 51.0]
total_rewards:  [47.0, 72.0, 81.0]
total_rewards:  [85.0, 58.0, 57.0]
total_rewards:  [94.0, 19.0, 70.0, 17.0]
total_rewards:  [20.0, 78.0, 46.0, 56.0]
total_rewards:  [65.0, 82.0, 53.0]
total_rewards:  [67.0, 98.0, 35.0]
total_rewards:  [31.0, 110.0, 59.0]
total_rewards:  [83.0, 117.0]
total_rewards:  [34.0, 64.0, 102.0]
total_rewards:  [86.0, 63.0, 51.0]
total_rewards:  [74.0, 126.0]
total_rewards:  [64.0, 97.0, 39.0]
total_rewards:  [110.0, 77.0, 13.0]
total_rewards:  [28.0, 86.0, 86.0]
total_rew

total_rewards:  [25.0, 38.0, 36.0, 33.0, 38.0, 30.0]
total_rewards:  [29.0, 47.0, 54.0, 52.0, 18.0]
total_rewards:  [34.0, 33.0, 47.0, 21.0, 50.0, 15.0]
total_rewards:  [49.0, 25.0, 61.0, 33.0, 32.0]
total_rewards:  [66.0, 53.0, 67.0, 14.0]
total_rewards:  [54.0, 91.0, 44.0, 11.0]
total_rewards:  [37.0, 85.0, 78.0]
total_rewards:  [96.0, 53.0, 47.0, 4.0]
total_rewards:  [54.0, 100.0, 46.0]
total_rewards:  [154.0, 46.0]
total_rewards:  [95.0, 62.0, 43.0]
total_rewards:  [48.0, 106.0, 46.0]
total_rewards:  [156.0, 44.0]
total_rewards:  [93.0, 73.0, 34.0]
total_rewards:  [96.0, 103.0, 1.0]
total_rewards:  [97.0, 97.0, 6.0]
total_rewards:  [132.0, 11.0, 57.0]
total_rewards:  [76.0, 60.0, 64.0]
total_rewards:  [97.0, 70.0, 20.0, 13.0]
total_rewards:  [55.0, 145.0]
total_rewards:  [77.0, 87.0, 36.0]
total_rewards:  [128.0, 18.0, 54.0]
total_rewards:  [116.0, 84.0]
total_rewards:  [168.0, 32.0]
total_rewards:  [124.0, 76.0]
total_rewards:  [112.0, 88.0]
total_rewards:  [200.0]
total_rewards: 

total_rewards:  [101.0, 57.0, 42.0]
total_rewards:  [130.0, 70.0]
total_rewards:  [108.0, 64.0, 28.0]
total_rewards:  [118.0, 80.0, 2.0]
total_rewards:  [59.0, 141.0]
total_rewards:  [93.0, 74.0, 33.0]
total_rewards:  [70.0, 130.0]
total_rewards:  [102.0, 78.0, 20.0]
total_rewards:  [101.0, 91.0, 8.0]
total_rewards:  [71.0, 86.0, 43.0]
total_rewards:  [85.0, 79.0, 36.0]
total_rewards:  [148.0, 52.0]
total_rewards:  [103.0, 97.0]
total_rewards:  [163.0, 37.0]
total_rewards:  [78.0, 100.0, 22.0]
total_rewards:  [79.0, 121.0]
total_rewards:  [194.0, 6.0]
total_rewards:  [104.0, 96.0]
total_rewards:  [144.0, 56.0]
total_rewards:  [122.0, 78.0]
total_rewards:  [100.0, 65.0, 35.0]
total_rewards:  [85.0, 91.0, 24.0]
total_rewards:  [106.0, 89.0, 5.0]
total_rewards:  [134.0, 66.0]
total_rewards:  [93.0, 85.0, 22.0]
total_rewards:  [76.0, 124.0]
total_rewards:  [67.0, 100.0, 33.0]
total_rewards:  [128.0, 72.0]
total_rewards:  [55.0, 84.0, 61.0]
total_rewards:  [97.0, 71.0, 32.0]
total_rewards: 

...

total_rewards:  [30.0, 153.0, 17.0]
total_rewards:  [80.0, 64.0, 56.0]
total_rewards:  [57.0, 48.0, 94.0, 1.0]
total_rewards:  [43.0, 107.0, 50.0]
total_rewards:  [92.0, 56.0, 52.0]
total_rewards:  [61.0, 137.0, 2.0]
total_rewards:  [80.0, 120.0]
total_rewards:  [114.0, 86.0]
total_rewards:  [111.0, 89.0]
total_rewards:  [133.0, 67.0]
total_rewards:  [84.0, 97.0, 19.0]
total_rewards:  [148.0, 52.0]
total_rewards:  [116.0, 84.0]
total_rewards:  [111.0, 89.0]
total_rewards:  [95.0, 105.0]
total_rewards:  [83.0, 117.0]
total_rewards:  [122.0, 78.0]
total_rewards:  [51.0, 149.0]
total_rewards:  [193.0, 7.0]
total_rewards:  [128.0, 72.0]
total_rewards:  [41.0, 159.0]
total_rewards:  [200.0]
total_rewards:  [113.0, 87.0]
total_rewards:  [167.0, 33.0]
total_rewards:  [199.0, 1.0]
total_rewards:  [200.0]
total_rewards:  [155.0, 45.0]
total_rewards:  [132.0, 68.0]
total_rewards:  [20.0, 121.0, 59.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [139.0, 

total_rewards:  [115.0, 60.0, 25.0]
total_rewards:  [64.0, 54.0, 63.0, 19.0]
total_rewards:  [79.0, 92.0, 29.0]
total_rewards:  [67.0, 47.0, 79.0, 7.0]
total_rewards:  [57.0, 110.0, 33.0]
total_rewards:  [55.0, 119.0, 26.0]
total_rewards:  [60.0, 75.0, 65.0]
total_rewards:  [84.0, 51.0, 65.0]
total_rewards:  [149.0, 51.0]
total_rewards:  [118.0, 82.0]
total_rewards:  [200.0]
total_rewards:  [168.0, 32.0]
total_rewards:  [84.0, 107.0, 9.0]
total_rewards:  [134.0, 66.0]
total_rewards:  [60.0, 140.0]
total_rewards:  [109.0, 91.0]
total_rewards:  [95.0, 105.0]
total_rewards:  [197.0, 3.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [166.0, 34.0]
total_rewards:  [117.0, 83.0]
total_rewards:  [102.0, 98.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [82.0, 118.0]
total_rewards:  [200.0]
total_rewards:  [194.0, 6.0]
total_rewards:  [200.0]
total_rewards:  [20

total_rewards:  [20.0, 11.0, 19.0, 10.0, 12.0, 10.0, 11.0, 13.0, 9.0, 9.0, 13.0, 15.0, 32.0, 14.0, 2.0]
total_rewards:  [10.0, 11.0, 18.0, 8.0, 11.0, 8.0, 12.0, 16.0, 11.0, 14.0, 25.0, 10.0, 12.0, 19.0, 14.0, 1.0]
total_rewards:  [22.0, 9.0, 12.0, 19.0, 13.0, 11.0, 11.0, 15.0, 33.0, 16.0, 18.0, 15.0, 6.0]
total_rewards:  [10.0, 12.0, 8.0, 11.0, 11.0, 10.0, 17.0, 21.0, 10.0, 18.0, 9.0, 16.0, 24.0, 13.0, 10.0]
total_rewards:  [23.0, 15.0, 20.0, 16.0, 14.0, 13.0, 12.0, 11.0, 11.0, 13.0, 27.0, 14.0, 11.0]
total_rewards:  [14.0, 10.0, 15.0, 22.0, 20.0, 14.0, 12.0, 16.0, 14.0, 10.0, 13.0, 19.0, 12.0, 9.0]
total_rewards:  [15.0, 12.0, 10.0, 13.0, 18.0, 14.0, 10.0, 16.0, 14.0, 16.0, 24.0, 28.0, 10.0]
total_rewards:  [16.0, 23.0, 16.0, 18.0, 16.0, 27.0, 13.0, 14.0, 12.0, 15.0, 18.0, 10.0, 2.0]
total_rewards:  [25.0, 14.0, 16.0, 10.0, 16.0, 19.0, 11.0, 23.0, 10.0, 22.0, 28.0, 6.0]
total_rewards:  [18.0, 14.0, 15.0, 11.0, 14.0, 20.0, 13.0, 13.0, 11.0, 15.0, 14.0, 15.0, 14.0, 13.0]
total_rewards: 

total_rewards:  [199.0, 1.0]
total_rewards:  [200.0]
total_rewards:  [145.0, 55.0]
total_rewards:  [192.0, 8.0]
total_rewards:  [132.0, 68.0]
total_rewards:  [134.0, 66.0]
total_rewards:  [159.0, 41.0]
total_rewards:  [36.0, 164.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [184.0, 16.0]
total_rewards:  [200.0]
total_rewards:  [144.0, 56.0]
total_rewards:  [185.0, 15.0]
total_rewards:  [200.0]
total_rewards:  [140.0, 60.0]
total_rewards:  [176.0, 24.0]
total_rewards:  [173.0, 27.0]
total_rewards:  [171.0, 29.0]
total_rewards:  [200.0]
total_rewards:  [22.0, 178.0]
total_rewards:  [189.0, 11.0]
total_rewards:  [165.0, 35.0]
total_rewards:  [175.0, 25.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [140.0, 60.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]


total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [160.0, 40.0]
total_rewards:  [200.0]
total_rewards:  [182.0, 18.0]
total_rewards:  [200.0]
total_rewards:  [26.0, 174.0]
total_rewards:  [184.0, 16.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [32.0, 168.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [93.0, 107.0]
total_rewards:  [68.0, 132.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [136.0, 64.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [200.0]
total_rewards:  [134.0, 66.0]
total_rewards:  [200.0]
total_rewa

In [11]:
observation = env.reset()
obs_vector = np.expand_dims(observation, axis=0)
obs_vector.shape


(1, 4)

In [17]:
(pl_calculated, pl_state, pl_actions,
    pl_advantages, pl_optimizer) = policy_grad

pl_calculated
   

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


probs = sess.run(
    pl_calculated,
    feed_dict={pl_state: obs_vector})

probs


array([[0.5079033 , 0.49209666]], dtype=float32)

In [23]:
pl_calculated.graph()

TypeError: 'Graph' object is not callable

In [20]:
obs_vector

array([[ 0.03047183,  0.00344319, -0.00064845,  0.01113388]])